# Testing the AI Service /crop Endpoint

This notebook serves as a test client for the `/crop` endpoint of the AI service. It will:
1. Load an image from a local file
2. Convert it to base64
3. Send it to the /crop endpoint
4. Display both the original and cropped images

In [ ]:
# Import necessary libraries
import requests
import base64
import json
import io
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Configuration
AI_SERVICE_URL = 'http://localhost:5050/crop'

# Create a test_images directory if it doesn't exist
if not os.path.exists('test_images'):
    os.makedirs('test_images')
    print("Created 'test_images' directory. Please put your test images there.")

In [ ]:
# Function to list all image files in the test_images directory
def list_image_files():
    if not os.path.exists('test_images'):
        return []
    
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    return [
        f for f in os.listdir('test_images') 
        if os.path.isfile(os.path.join('test_images', f)) and 
        any(f.lower().endswith(ext) for ext in image_extensions)
    ]

# List available image files
image_files = list_image_files()
if image_files:
    print("Available test images:")
    for i, file in enumerate(image_files):
        print(f"{i+1}. {file}")
else:
    print("No image files found in 'test_images' directory. Please add some test images.")

In [ ]:
# Function to load an image and convert it to base64
def image_to_base64(file_path):
    with open(file_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# Function to convert base64 back to an image
def base64_to_image(base64_string):
    image_data = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(image_data))

In [ ]:
# Select an image to test (change the index as needed)
image_index = 0  # Change this to select a different image from the available list

if image_files:
    selected_image = image_files[image_index]
    image_path = os.path.join('test_images', selected_image)
    print(f"Selected image: {selected_image}")
    
    # Display the original image
    original_img = Image.open(image_path)
    plt.figure(figsize=(10, 8))
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis('off')
    plt.show()
    
    # Convert the image to base64
    base64_image = image_to_base64(image_path)
    print(f"Image converted to base64 (length: {len(base64_image)} characters)")
else:
    print("Please add images to the 'test_images' directory before running this cell.")

In [ ]:
# Function to send the image to the /crop endpoint and process the response
def crop_image(base64_image):
    payload = {"image": base64_image}
    
    try:
        response = requests.post(AI_SERVICE_URL, json=payload)
        
        # Print response status code and content length
        print(f"Response status code: {response.status_code}")
        print(f"Response content length: {len(response.content)} bytes")
        
        if response.status_code == 200:
            result = response.json()
            return result
        else:
            print(f"Error: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

In [ ]:
# Send the image to the /crop endpoint and display the results
if 'base64_image' in locals():
    # Call the crop endpoint
    print("Sending image to /crop endpoint...")
    result = crop_image(base64_image)
    
    if result and result.get('success'):
        # Extract information from the response
        cropped_base64 = result.get('cropped_image')
        cover_type = result.get('cover_type')
        message = result.get('message', 'No message provided')
        
        print(f"Cover type detected: {cover_type}")
        print(f"Message: {message}")
        
        # Convert the cropped image back from base64
        cropped_img = base64_to_image(cropped_base64)
        
        # Display the cropped image
        plt.figure(figsize=(10, 8))
        plt.imshow(cropped_img)
        plt.title(f"Cropped Image (Cover Type: {cover_type})")
        plt.axis('off')
        plt.show()
        
        # Display original and cropped side by side for comparison
        plt.figure(figsize=(20, 10))
        
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original Image")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(cropped_img)
        plt.title(f"Cropped Image (Cover Type: {cover_type})")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Save the cropped image to file (optional)
        output_dir = "test_results"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        output_filename = f"{os.path.splitext(selected_image)[0]}_cropped.jpg"
        output_path = os.path.join(output_dir, output_filename)
        cropped_img.save(output_path)
        print(f"Cropped image saved to {output_path}")
    else:
        print("Failed to crop the image or API call unsuccessful")
else:
    print("Please run the previous cells to load an image first.")

## Test Multiple Images (Optional)

The code below allows you to test multiple images in batch mode.

In [ ]:
# Test all images in the folder
def test_all_images():
    image_files = list_image_files()
    if not image_files:
        print("No images found in 'test_images' directory.")
        return
    
    for image_file in image_files:
        print(f"\nProcessing {image_file}...")
        image_path = os.path.join('test_images', image_file)
        
        # Convert the image to base64
        base64_image = image_to_base64(image_path)
        
        # Call the crop endpoint
        result = crop_image(base64_image)
        
        if result and result.get('success'):
            print(f"Cover type: {result.get('cover_type')}")
            print(f"Message: {result.get('message', 'No message')}")
        else:
            print("Failed to crop the image")

# Uncomment the following line to test all images
# test_all_images()